In [9]:
# Librerías a importar
from pyspark.sql import SparkSession
from pyspark.sql.functions import col # Función que nos sirve para seleccionar columnas

In [10]:
# Crear la sesión de spark
spark = SparkSession  \
    .builder \
    .appName("La cantidad de libros vendidos") \
    .config('spark.driver.extraClassPath', "mysql-connector-j-8.0.32.jar") \
    .getOrCreate()

# Propiedades para la conexión usando spark jdbc
url = "jdbc:mysql://localhost:3306/libreria"
propiedades = {
    "driver": "com.mysql.cj.jdbc.Driver",
    "user": "root",
    "password": "tooR"
}

In [13]:
# Leemos la tabla detalle_factura de la base de datos librería, acá está la información de la cantidad vendida.
detalle_factura_df = spark.read.jdbc(
    url=url, table="detalle_factura", properties=propiedades)

# Agrupamos la cantidad de ventas por el id del libro
cantidad_libros_vendidos_df = detalle_factura_df.groupBy("id_libro").sum("cantidad")

# Mostramos la cantidad de libros vendidos por el id del libro
cantidad_libros_vendidos_df.show()

+--------+-------------+
|id_libro|sum(cantidad)|
+--------+-------------+
|      31|            3|
|      34|            8|
|      28|           15|
|      27|            3|
|      26|            5|
|      44|            5|
|      12|           10|
|      22|            6|
|      47|           10|
|       1|           10|
|      13|           10|
|       6|            7|
|      16|            9|
|       3|            4|
|      40|            3|
|      20|            4|
|       5|            5|
|      19|            3|
|      41|            2|
|      43|            5|
+--------+-------------+
only showing top 20 rows



In [12]:


# Leemos la tabla libro de la base de datos librería
libro_df = spark.read.jdbc(url=url, table="libro", properties=propiedades)

# Unimos el dataframe cantidad_libros_vendidos_df con la tabla de libro por medio del id del libro
cantidad_libros_vendidos_df = cantidad_libros_vendidos_df.join(
    libro_df,
    libro_df.id == cantidad_libros_vendidos_df.id_libro)

# Seleccionamos solo la información necesaria: título, puntación y cantidad vendida.
cantidad_libros_vendidos_df = cantidad_libros_vendidos_df.select(
    col("titulo"),
    col("puntuacion"),
    col("sum(cantidad)").alias("cantidad_vendidos")
).orderBy("cantidad_vendidos", ascending=True)

# Mostramos el dataframe cantidad_libros_vendidos_df
cantidad_libros_vendidos_df.show()

+--------------------+----------+-----------------+
|              titulo|puntuacion|cantidad_vendidos|
+--------------------+----------+-----------------+
|Amigo de los perd...|      2.75|                2|
|    Honor del océano|      4.17|                2|
|  Enemigos con vigor|      3.50|                2|
|     Paz a la nación|      2.00|                2|
|Canciones de mi f...|      4.84|                3|
|Defensores y hombres|      1.18|                3|
|  Unidad de grandeza|      2.33|                3|
| Perros de esperanza|      2.08|                3|
|       Raíz de plata|      4.72|                3|
|     Héroes del ayer|      1.45|                3|
|      León del dolor|      2.76|                4|
| Gritos en mi pasado|      2.29|                4|
| Aquel primer verano|      4.17|                4|
| Señales en el norte|      4.20|                4|
|     Amigo de la luz|      3.98|                4|
|Ascendencia de lo...|      2.05|                5|
|     Ayudan